In [1]:
import csv
import numpy
import pandas as pd
import math


def editDistance(r, h):
    '''
    This function is to calculate the edit distance of reference sentence and the hypothesis sentence.

    Main algorithm used is dynamic programming.

    Attributes: 
        r -> the list of words produced by splitting reference sentence.
        h -> the list of words produced by splitting hypothesis sentence.
    '''
    d = numpy.zeros((len(r)+1)*(len(h)+1), dtype=numpy.uint8).reshape((len(r)+1, len(h)+1))
    for i in range(len(r)+1):
        d[i][0] = i
    for j in range(len(h)+1):
        d[0][j] = j
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                substitute = d[i-1][j-1] + 1
                insert = d[i][j-1] + 1
                delete = d[i-1][j] + 1
                d[i][j] = min(substitute, insert, delete)
    return d





In [2]:
#load excel results
ali = pd.read_excel("ali_api_audio_result.xlsx")
baidu = pd.read_excel("baidu_api_audio_result(1).xlsx")
IBM = pd.read_excel("IBM_api_audio_result.xlsx")

In [68]:
micro = pd.read_excel("Microsoft_api_audio_result840.xlsx")

In [3]:
ali = ali.sort_values(by=["file_name"]).reset_index(drop=True).applymap(lambda s:s.lower() if type(s) == str else s)
ali = ali.rename(columns={"result":"ali"})
baidu = baidu.sort_values(by=["file_name"]).reset_index(drop=True).applymap(lambda s:s.lower() if type(s) == str else s)
baidu = baidu.rename(columns={"result":"baidu"})
IBM = IBM.sort_values(by=["file_name"]).reset_index(drop=True).applymap(lambda s:s.lower() if type(s) == str else s)
IBM = IBM.rename(columns={"result":"IBM"})

In [69]:
micro = micro.sort_values(by=["file_name"]).reset_index(drop=True).applymap(lambda s:s.lower() if type(s) == str else s)
micro = micro.rename(columns={"result":"micro"})

In [70]:
#load original product
txt = pd.read_fwf("newtext.txt",header=None).applymap(lambda s:s.lower() if type(s) == str else s)

In [13]:
combine = (txt[1].str.replace('.', '').str.strip()).to_frame()
combine = combine.join([ali["ali"].str.replace('.', '').str.strip()])
combine = combine.join([baidu["baidu"].str.replace('.', '').str.strip()])
combine = combine.join([IBM["IBM"].str.replace('.', '').str.strip()])

NameError: name 'ali' is not defined

In [71]:
combine = (txt[1].str.replace('.', '').str.strip()).to_frame()
combine = combine.join([micro["micro"].str.replace('.', '').str.strip()])

In [72]:
combine = combine.dropna()

In [7]:
AliDiff = 0
AliTotal = 0
BaiduDiff = 0
BaiduTotal = 0
IBMDiff = 0
IBMTotal = 0

In [73]:
microDiff = 0
microTotal = 0

In [74]:
for index, row in combine.iterrows():
    r = row[1]
    h = row["micro"]
    d = editDistance(r, h)
    
    microDiff += d[len(r)][len(h)]
    microTotal += len(r)
    
WER_micro = (microDiff/microTotal*100)

In [75]:
WER_micro

2.081719597211464

In [8]:
for index, row in combine.iterrows():
    r = row[1]
    h = row["ali"]
    d = editDistance(r, h)
    
    AliDiff += d[len(r)][len(h)]
    AliTotal += len(r)
    
WER_Ali = (AliDiff/AliTotal*100)

In [9]:
for index, row in combine.iterrows():
    r = row[1]
    h = row["baidu"]
    d = editDistance(r, h)
    
    BaiduDiff += d[len(r)][len(h)]
    BaiduTotal += len(r)
    
WER_baidu = (BaiduDiff/BaiduTotal*100)

In [10]:
for index, row in combine.iterrows():
    r = row[1]
    h = row["IBM"]
    d = editDistance(r, h)
    
    IBMDiff += d[len(r)][len(h)]
    IBMTotal += len(r)
    
WER_IBM = (IBMDiff/IBMTotal*100)

In [11]:
WER_Ali

1.2347040017638629

In [12]:
WER_baidu

1.796935288281336

In [13]:
WER_IBM

2.1662440745232057